<a href="https://colab.research.google.com/github/Pranav-M23/Car_Assistant_LLM/blob/main/Pranav_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, torch, gc
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

Sat Feb 28 13:58:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip -q install -U transformers accelerate datasets peft trl bitsandbytes safetensors

In [ ]:
!ls -lh /content | head



In [ ]:
from datasets import load_dataset

# Update 'data_files' with the correct path to your train.jsonl file.
# For example, if it's in your Google Drive, it might look like this:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/mini/content/train.jsonl", split="train") # <-- PLEASE UPDATE THIS LINE WITH YOUR ACTUAL PATH
# dataset = load_dataset("json", data_files="train.jsonl", split="train")
print("Dataset size:", len(dataset))
print(dataset[0])

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset

# -----------------------------
# CONFIG
# -----------------------------
MODEL_NAME = "microsoft/phi-2"   # SAFE for Colab T4
DATA_PATH = "/content/drive/MyDrive/mini/content/train.jsonl"
OUT_DIR = "/content/car-assistant-qlora"

# -----------------------------
# LOAD DATASET
# -----------------------------
dataset = load_dataset("json", data_files=DATA_PATH, split="train")
print("Dataset size:", len(dataset))

# -----------------------------
# TOKENIZER
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 🔑 IMPORTANT: set sequence length HERE
tokenizer.model_max_length = 384

# -----------------------------
# QLoRA CONFIG
# -----------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# -----------------------------
# MODEL
# -----------------------------
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True
)
model.config.use_cache = False

# -----------------------------
# LoRA CONFIG
# -----------------------------
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj"]
)

args = TrainingArguments(
    output_dir=OUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=1,

    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    report_to="none"
)



def formatting_func(example):
    return example["text"]


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=args,
    formatting_func=formatting_func,
    processing_class=tokenizer
)


# TRAIN
# -----------------------------
trainer.train()
trainer.save_model(OUT_DIR)

print("✅ Fine-tuning completed.")
print("✅ LoRA adapter saved at:", OUT_DIR)


In [ ]:
!ls -lh /content
!ls -lh /content/car-assistant-qlora | head


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

MODEL_NAME = "microsoft/phi-2"
OUT_DIR = "/content/car-assistant-qlora"   # make sure this folder exists

# 4-bit quant config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Base model
base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True
)
base.eval()

# Attach LoRA adapter
ft = PeftModel.from_pretrained(base, OUT_DIR)
ft.eval()

# Prompt
prompt = "<s>[INST] My car makes a clicking sound while starting. No warning lights. Provide possible causes and safe advice. [/INST]"

# ✅ Tokenize -> generate -> decode
inputs = tokenizer(prompt, return_tensors="pt").to(ft.device)

with torch.no_grad():
    output_ids = ft.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

# Optional: keep only assistant part
if "[/INST]" in text:
    text = text.split("[/INST]", 1)[-1].strip()

print(text.replace("</s>", "").strip())



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!zip -r car_assistant_adapter.zip /content/car-assistant-qlora


In [4]:
!pip -q install -U gradio transformers accelerate peft bitsandbytes safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 10.8 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/mini/content
!ls -lah /content/drive/MyDrive/mini/content/car-assistant-qlora


In [3]:
import re

OBD_RE = re.compile(r"\bP[0-3][0-9A-F]{3}\b", re.IGNORECASE)

OBD_HINTS = {
    "P0300": "Random/Multiple Cylinder Misfire Detected",
    "P0301": "Cylinder 1 Misfire Detected",
    "P0171": "System Too Lean (Bank 1)",
    "P0420": "Catalyst System Efficiency Below Threshold (Bank 1)",
    "P0128": "Coolant Thermostat (Coolant Temp Below Regulating Temp)",
    "P0455": "EVAP System Large Leak Detected",
    "P0700": "Transmission Control System Malfunction",
}

def normalize_obd_codes(obd_text: str):
    return sorted(set(m.group(0).upper() for m in OBD_RE.finditer(obd_text or "")))

def explain_obd(codes):
    if not codes:
        return "None provided."
    lines = []
    for c in codes:
        lines.append(f"{c}: {OBD_HINTS.get(c, 'Unknown/General OBD-II code (lookup needed)')}")
    return "\n".join(lines)

print("✅ OBD helper ready.")


✅ OBD helper ready.


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

MODEL_NAME = "microsoft/phi-2"
ADAPTER_DIR = "/content/drive/MyDrive/mini/content/car-assistant-qlora"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True
)
base_model.eval()
model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)

model.eval()

print("✅ LoRA adapter attached. Fine-tuned model ready.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ LoRA adapter attached. Fine-tuned model ready.


In [6]:
import torch, re

REQUIRED_HEADINGS = [
    "Symptom Summary:",
    "OBD-II Interpretation:",
    "Likely Causes",
    "Risk Level",
    "Safe Checks",
    "Do NOT Do",
    "Next Action"
]

def looks_valid(text: str) -> bool:
    t = text.strip()
    return sum(h.lower() in t.lower() for h in REQUIRED_HEADINGS) >= 4  # at least 4 headings appear

# Ban common code-y starts (helps a LOT)
bad_words = ["def ", "class ", "import ", "argparse", "parser =", "raw_input", "print(", "super().__init__", "pydantic"]
bad_words_ids = [tokenizer(bw, add_special_tokens=False).input_ids for bw in bad_words if len(tokenizer(bw, add_special_tokens=False).input_ids) > 0]

SYSTEM_FORMAT = """Return output in this exact format (no code, no HTML):

Symptom Summary:
OBD-II Interpretation:
Likely Causes (ranked):
Risk Level (Low/Medium/High):
Safe Checks (user can do):
Do NOT Do:
Next Action:
"""

def generate_once(symptoms: str, obd_text: str, temperature: float):
    codes = normalize_obd_codes(obd_text or "")
    codes_str = ", ".join(codes) if codes else "None"
    obd_info = explain_obd(codes)

    prompt = (
        f"<s>[INST] You are a car diagnostic assistant. "
        f"IMPORTANT: Do NOT write code. Do NOT write HTML. "
        f"Only write the sections exactly as specified.\n\n"
        f"{SYSTEM_FORMAT}\n"
        f"Symptoms: {symptoms.strip()}\n"
        f"OBD Codes: {codes_str}\n"
        f"OBD Notes:\n{obd_info}\n"
        f"[/INST]\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            repetition_penalty=1.25,
            bad_words_ids=bad_words_ids,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    text = tokenizer.decode(out_ids[0], skip_special_tokens=False)
    if "[/INST]" in text:
        text = text.split("[/INST]", 1)[-1]
    return text.replace("</s>", "").replace("<s>", "").strip()

def baseline_fallback(symptoms: str, obd_text: str):
    codes = normalize_obd_codes(obd_text or "")
    codes_str = ", ".join(codes) if codes else "None"

    # Expand OBD hints a bit
    local_obd = dict(OBD_HINTS)
    local_obd.update({
        "P0302": "Cylinder 2 Misfire Detected",
        "P0303": "Cylinder 3 Misfire Detected",
        "P0304": "Cylinder 4 Misfire Detected",
        "P0351": "Ignition Coil A Primary/Secondary Circuit",
        "P0352": "Ignition Coil B Primary/Secondary Circuit",
        "P0117": "Engine Coolant Temp Circuit Low Input",
        "P0118": "Engine Coolant Temp Circuit High Input",
    })

    def explain_obd2(codes):
        if not codes:
            return "None provided."
        return "\n".join([f"{c}: {local_obd.get(c, 'Unknown/General OBD-II code (lookup needed)')}" for c in codes])

    obd_info = explain_obd2(codes)

    s = (symptoms or "").lower()

    # Risk heuristic
    high_risk_terms = ["brake", "overheat", "steam", "smoke", "fuel smell", "petrol smell", "raw gas", "burning smell"]
    risk = "High" if any(t in s for t in high_risk_terms) else "Medium"

    # --- Rule-based likely causes ---
    causes = []
    safe_checks = []
    do_not = []
    next_action = []

    # Fuel smell + shaking -> possible misfire + fuel leak
    if ("fuel smell" in s or "petrol smell" in s or "raw gas" in s) and ("shake" in s or "shaking" in s or "rough" in s):
        causes = [
            "Misfire causing unburnt fuel (spark plug/ignition coil/injector)",
            "Fuel leak in engine bay or EVAP system leak",
            "Air-fuel imbalance (MAF/MAP sensor, vacuum leak)"
        ]
        safe_checks = [
            "If smell is strong, turn off engine and check for visible leaks under hood/near fuel lines (no flames).",
            "Check if Check Engine Light is flashing (flashing = stop driving).",
            "Note whether shaking happens at idle only or also while driving."
        ]
        do_not = [
            "Do not continue driving if fuel smell is strong or if the engine is shaking heavily.",
            "Do not smoke or use open flame near the vehicle."
        ]
        next_action = [
            "Tow to a service center if fuel smell is strong.",
            "Ask for misfire diagnosis (spark plugs, coils, injectors) and fuel leak inspection."
        ]
        risk = "High"

    # Clicking on start -> battery/starter
    elif ("click" in s or "clicking" in s) and ("start" in s or "starting" in s or "crank" in s):
        causes = [
            "Weak battery or loose/corroded terminals",
            "Starter relay/solenoid issue",
            "Starter motor issue"
        ]
        safe_checks = [
            "Check battery terminals for looseness/corrosion (engine OFF).",
            "Observe if headlights dim a lot during crank attempt.",
            "Try jump start if available (follow safe procedure)."
        ]
        do_not = [
            "Do not crank repeatedly for long durations.",
            "Do not ignore burning smell or smoke."
        ]
        next_action = [
            "Get battery + starter + charging system tested.",
            "If it starts with jump, battery/charging is likely."
        ]

    # Overheating -> cooling system
    elif ("overheat" in s or "temperature" in s or "temp" in s or "steam" in s):
        causes = [
            "Low coolant / coolant leak",
            "Thermostat stuck",
            "Radiator fan not working",
            "Water pump issue"
        ]
        safe_checks = [
            "Stop driving if gauge goes to red; let engine cool.",
            "Check coolant level only when engine is cool.",
            "Look for coolant leaks under the car."
        ]
        do_not = [
            "Do not open radiator cap when hot.",
            "Do not continue driving while overheating."
        ]
        next_action = [
            "Cooling system inspection recommended immediately.",
            "Tow if overheating persists."
        ]
        risk = "High"

    # Brakes -> braking system
    elif "brake" in s or "braking" in s or "grinding" in s or "soft pedal" in s:
        causes = [
            "Worn brake pads/rotors",
            "Brake fluid leak / air in brake lines",
            "Stuck caliper"
        ]
        safe_checks = [
            "Check brake fluid level if accessible.",
            "Test braking gently at low speed in a safe area."
        ]
        do_not = [
            "Do not drive at high speed.",
            "Do not ignore grinding noises."
        ]
        next_action = [
            "Brake inspection urgently recommended."
        ]
        risk = "High"

    # Misfire codes -> engine misfire bucket
    elif any(c.startswith("P03") for c in codes) or ("misfire" in s or "rough idle" in s or "shaking" in s):
        causes = [
            "Spark plug worn/fouled",
            "Ignition coil issue",
            "Fuel injector issue",
            "Vacuum leak / MAF sensor issue"
        ]
        safe_checks = [
            "If Check Engine Light is flashing, avoid driving.",
            "Note if issue worsens under acceleration.",
            "Check for loose intake hose if visible."
        ]
        do_not = [
            "Do not keep driving long distances with severe misfire."
        ]
        next_action = [
            "Request misfire diagnosis (plugs/coils/injectors) and read freeze-frame data."
        ]
        risk = "High" if "flashing" in s else risk

    # Default generic
    else:
        causes = [
            "General engine/electrical issue",
            "Sensor or maintenance-related fault"
        ]
        safe_checks = [
            "Note when the symptom occurs (startup/idle/acceleration).",
            "Provide vehicle model/year and any warning lights.",
            "Scan OBD-II codes if possible."
        ]
        do_not = [
            "Do not ignore persistent warning lights."
        ]
        next_action = [
            "If persistent, visit a technician with the symptom notes."
        ]

    # Build response in your required format
    causes_fmt = "\n".join([f"{i+1}) {c}" for i, c in enumerate(causes)])
    checks_fmt = "\n".join([f"- {c}" for c in safe_checks])
    donot_fmt = "\n".join([f"- {d}" for d in do_not])
    next_fmt = "\n".join([f"- {n}" for n in next_action])

    return f"""Symptom Summary:
- {symptoms.strip()}

OBD-II Interpretation:
- Codes: {codes_str}
{obd_info}

Likely Causes (ranked):
{causes_fmt}

Risk Level (Low/Medium/High):
- {risk}

Safe Checks (user can do):
{checks_fmt}

Do NOT Do:
{donot_fmt}

Next Action:
{next_fmt}
"""


def diagnose(symptoms: str, obd_text: str):
    symptoms = (symptoms or "").strip()
    if not symptoms:
        return "Please enter symptoms (e.g., clicking sound when starting, warning light, vibration, smell)."


    out1 = generate_once(symptoms, obd_text, temperature=0.5)
    if looks_valid(out1):
        return out1


    out2 = generate_once(symptoms, obd_text, temperature=0.2)
    if looks_valid(out2):
        return out2


    return baseline_fallback(symptoms, obd_text)

print("✅ Guardrailed diagnose() ready.")
print(diagnose("Clicking sound when starting, no warning lights", "P0300"))
print("\n" + "="*60 + "\n")
print(diagnose("Strong raw gas smell and heavy engine shaking", "P0302"))




✅ Guardrailed diagnose() ready.
Symptom Summary:
- Clicking sound when starting, no warning lights

OBD-II Interpretation:
- Codes: P0300
P0300: Random/Multiple Cylinder Misfire Detected

Likely Causes (ranked):
1) Weak battery or loose/corroded terminals
2) Starter relay/solenoid issue
3) Starter motor issue

Risk Level (Low/Medium/High):
- Medium

Safe Checks (user can do):
- Check battery terminals for looseness/corrosion (engine OFF).
- Observe if headlights dim a lot during crank attempt.
- Try jump start if available (follow safe procedure).

Do NOT Do:
- Do not crank repeatedly for long durations.
- Do not ignore burning smell or smoke.

Next Action:
- Get battery + starter + charging system tested.
- If it starts with jump, battery/charging is likely.



Symptom Summary:
- Strong raw gas smell and heavy engine shaking

OBD-II Interpretation:
- Codes: P0302
P0302: Cylinder 2 Misfire Detected

Likely Causes (ranked):
1) Misfire causing unburnt fuel (spark plug/ignition coil/injec

In [ ]:
import gradio as gr


EXAMPLES = [
    ["Clicking sound when starting, no warning lights", "P0300"],
    ["Strong raw gas smell and heavy engine shaking", "P0302"],
    ["Engine overheating, temperature gauge goes high, steam near hood", "P0128"],
    ["Grinding noise while braking, brake pedal feels soft", ""],
]

with gr.Blocks(title="Car Assistant LLM — Diagnosis + OBD-II") as demo:
    gr.Markdown("# 🚗 Car Assistant LLM — Problem Diagnosis & Guidance")
    gr.Markdown(
        "Enter your car problem systems and  OBD-II code(e.g., P0300). "
        "Our assistant shall return a  structured diagnosis and safety guidance.Please Follow the Guidelines Carefully"

    )

    with gr.Row():
        symptoms_in = gr.Textbox(
            lines=4,
            label="Symptoms ",
            placeholder="e.g., Clicking sound when starting, no warning lights"
        )
        obd_in = gr.Textbox(
            lines=1,
            label="OBD-II Code",
            placeholder="e.g., P0300, P0171"
        )

    diagnose_btn = gr.Button("Diagnose ✅")
    output_box = gr.Textbox(lines=16, label="Diagnosis & Guidance ")

    diagnose_btn.click(fn=diagnose, inputs=[symptoms_in, obd_in], outputs=output_box)

    gr.Markdown("## Quick Examples (for demo)")
    gr.Examples(
        examples=EXAMPLES,
        inputs=[symptoms_in, obd_in]
    )

    with gr.Row():
        clear_btn = gr.Button("Clear")
        clear_btn.click(lambda: ("", ""), outputs=[symptoms_in, obd_in])

demo.launch(share=True, debug=False)


In [ ]:
import gradio as gr

# =========================
# LOGIN CONFIG (DEMO ONLY)
# =========================
VALID_USERS = {
    "admin": "car123",     # username: password
    "sir": "demo123",      # optional extra
}

def do_login(username, password):
    u = (username or "").strip()
    p = (password or "").strip()
    if u in VALID_USERS and VALID_USERS[u] == p:
        return (
            gr.update(visible=False),  # hide login page
            gr.update(visible=True),   # show dashboard
            f"✅ Welcome, **{u}**!"
        )
    return (
        gr.update(visible=True),
        gr.update(visible=False),
        "❌ Invalid username or password."
    )

# =========================
# SAFE WRAPPER (in case diagnose missing)
# =========================
def safe_diagnose(symptoms, obd_text):
    if "diagnose" not in globals():
        return (
            "❌ diagnose() function not found.\n\n"
            "Paste your Step-7 diagnose(symptoms, obd_text) code ABOVE this UI block."
        )
    return diagnose(symptoms, obd_text)  # noqa

# =========================
# OBD CODE DATABASE (DEMO)
# Later replace with full CSV database
# =========================
OBD_DB = {
    "P0300": "Random/Multiple Cylinder Misfire Detected",
    "P0301": "Cylinder 1 Misfire Detected",
    "P0302": "Cylinder 2 Misfire Detected",
    "P0303": "Cylinder 3 Misfire Detected",
    "P0304": "Cylinder 4 Misfire Detected",
    "P0171": "System Too Lean (Bank 1)",
    "P0172": "System Too Rich (Bank 1)",
    "P0420": "Catalyst System Efficiency Below Threshold (Bank 1)",
    "P0128": "Coolant Thermostat (Coolant Temp Below Regulating Temperature)",
    "P0455": "EVAP System Leak Detected (Gross Leak)",
}

def obd_list_text():
    return "\n".join(sorted(OBD_DB.keys()))

def obd_lookup(code):
    code = (code or "").strip().upper()
    if not code:
        return "Enter an OBD code like P0300."
    if code in OBD_DB:
        return f"{code}: {OBD_DB[code]}"
    return (
        f"{code}: Not found in demo list.\n"
        "This may be manufacturer-specific (often P1xxx) or not included.\n"
        "Tip: Provide make/model/year for better accuracy."
    )

def obd_search(query):
    q = (query or "").strip().upper()
    if not q:
        return obd_list_text()
    matches = [c for c in sorted(OBD_DB.keys()) if q in c or q in OBD_DB[c].upper()]
    return "\n".join(matches) if matches else "No matches found."

# =========================
# COMMON CAR PROBLEMS (STATIC PAGE)
# You can expand this list
# =========================
COMMON_PROBLEMS = [
    ("Clicking sound on start", "Battery weak / terminals loose / starter relay issue"),
    ("Engine overheating", "Low coolant / radiator fan / thermostat / water pump"),
    ("Car pulls while braking", "Uneven pads / stuck caliper / alignment"),
    ("White smoke from exhaust", "Coolant leak into combustion (head gasket risk)"),
    ("Vibration at high speed", "Wheel balancing / alignment / worn tires"),
    ("Poor mileage", "Low tire pressure / dirty air filter / O2 sensor / driving style"),
]

def common_problems_text():
    return "\n\n".join([f"• {t}\n  - {d}" for t, d in COMMON_PROBLEMS])

# =========================
# WARNING LIGHT GUIDE (TEXT)
# Images can be added later as gallery
# =========================
WARNING_LIGHTS = [
    ("Check Engine (MIL)", "Scan OBD codes. If flashing: avoid driving, possible misfire."),
    ("Oil Pressure", "Stop engine ASAP. Low oil pressure can cause severe engine damage."),
    ("Battery/Charging", "Charging system fault; vehicle may stall soon."),
    ("Coolant Temperature", "Overheating risk. Stop and cool down; check coolant when safe."),
    ("ABS", "ABS may be disabled; braking control reduced. Service soon."),
    ("Airbag/SRS", "Airbag system fault. Safety risk; service recommended."),
]

def warning_lights_text():
    return "\n\n".join([f"• {n}\n  - {d}" for n, d in WARNING_LIGHTS])

# =========================
# PAGE SWITCHING LOGIC
# =========================
PAGES = ["assistant", "obd", "lights", "common"]

def show_page(page_name):
    # returns visibility updates for each page group
    return (
        gr.update(visible=(page_name == "assistant")),
        gr.update(visible=(page_name == "obd")),
        gr.update(visible=(page_name == "lights")),
        gr.update(visible=(page_name == "common")),
    )

# =========================
# UI
# =========================
with gr.Blocks(title="Car Assistant LLM — Dashboard") as app:
    gr.Markdown("# 🚗 Car Assistant LLM — Dashboard")

    # -------- LOGIN PAGE --------
    login_page = gr.Group(visible=True)
    with login_page:
        gr.Markdown("## 🔐 Login")
        with gr.Row():
            with gr.Column(scale=1):
                username = gr.Textbox(label="Username", placeholder="e.g., admin")
                password = gr.Textbox(label="Password", type="password", placeholder="e.g., car123")
                login_btn = gr.Button("Login")
                login_msg = gr.Markdown("")
            with gr.Column(scale=1):
                gr.Markdown(
                    "### Demo Credentials\n"
                    "- **admin / car123**\n"
                    "- **sir / demo123**\n"
                )

    # -------- DASHBOARD --------
    dashboard = gr.Group(visible=False)
    with dashboard:
        header_msg = gr.Markdown("")

        with gr.Row():
            # LEFT SIDEBAR
            with gr.Column(scale=1, min_width=220):
                gr.Markdown("## 📌 Sections")
                btn_assistant = gr.Button("🧰 Car Assistant")
                btn_obd = gr.Button("🔎 OBD Code Checker")
                btn_lights = gr.Button("🚨 Warning Light Guide")
                btn_common = gr.Button("🛠 Common Car Problems")
                gr.Markdown("---")
                gr.Markdown("Tip: For best results, add **make/model/year**.")

            # RIGHT CONTENT AREA
            with gr.Column(scale=4):
                # PAGE 1: Assistant
                page_assistant = gr.Group(visible=True)
                with page_assistant:
                    gr.Markdown("## 🧰 Car Assistant (Symptoms → Guidance)")
                    symptoms_in = gr.Textbox(lines=5, label="Symptoms", placeholder="e.g., Clicking sound when starting, no warning lights")
                    obd_in = gr.Textbox(lines=1, label="OBD Codes (optional)", placeholder="e.g., P0300, P0171")
                    run_btn = gr.Button("Diagnose ✅")
                    diag_out = gr.Textbox(lines=16, label="Diagnosis & Guidance")
                    run_btn.click(fn=safe_diagnose, inputs=[symptoms_in, obd_in], outputs=diag_out)

                    gr.Markdown("### Quick Examples")
                    gr.Examples(
                        examples=[
                            ["Clicking sound when starting, no warning lights", "P0300"],
                            ["Strong raw gas smell and heavy engine shaking", "P0302"],
                            ["Engine overheating, steam near hood", "P0128"],
                            ["Grinding noise while braking, pedal feels soft", ""],
                        ],
                        inputs=[symptoms_in, obd_in],
                    )

                # PAGE 2: OBD
                page_obd = gr.Group(visible=False)
                with page_obd:
                    gr.Markdown("## 🔎 OBD Code Checker")
                    with gr.Row():
                        with gr.Column():
                            search_q = gr.Textbox(label="Search (code or keyword)", placeholder="e.g., misfire / P03")
                            search_btn = gr.Button("Search")
                            codes_box = gr.Textbox(lines=12, value=obd_list_text(), label="Available Codes (Demo List)")
                            search_btn.click(fn=obd_search, inputs=search_q, outputs=codes_box)
                        with gr.Column():
                            code_in = gr.Textbox(label="Lookup Code", placeholder="P0302")
                            lookup_btn = gr.Button("Lookup")
                            code_out = gr.Textbox(lines=6, label="Meaning")
                            lookup_btn.click(fn=obd_lookup, inputs=code_in, outputs=code_out)

                # PAGE 3: Warning lights
                page_lights = gr.Group(visible=False)
                with page_lights:
                    gr.Markdown("## 🚨 Warning Light Guide")
                    lights_out = gr.Textbox(lines=18, value=warning_lights_text(), label="Meaning & Safe Action")

                # PAGE 4: Common problems
                page_common = gr.Group(visible=False)
                with page_common:
                    gr.Markdown("## 🛠 Common Car Problems")
                    common_out = gr.Textbox(lines=18, value=common_problems_text(), label="Quick Reference")

        # Wire sidebar buttons to page switching
        btn_assistant.click(fn=lambda: show_page("assistant"), inputs=None,
                           outputs=[page_assistant, page_obd, page_lights, page_common])
        btn_obd.click(fn=lambda: show_page("obd"), inputs=None,
                     outputs=[page_assistant, page_obd, page_lights, page_common])
        btn_lights.click(fn=lambda: show_page("lights"), inputs=None,
                        outputs=[page_assistant, page_obd, page_lights, page_common])
        btn_common.click(fn=lambda: show_page("common"), inputs=None,
                        outputs=[page_assistant, page_obd, page_lights, page_common])

    # Login wiring
    login_btn.click(fn=do_login, inputs=[username, password], outputs=[login_page, dashboard, header_msg])

app.launch(share=True)

In [7]:
import gradio as gr
import pandas as pd
import os

# =========================================================
# 0) LOGIN CONFIG (demo)
# =========================================================
VALID_USERS = {
    "admin": "car123",
    "sir": "demo123",
}

def do_login(username, password):
    u = (username or "").strip()
    p = (password or "").strip()
    if u in VALID_USERS and VALID_USERS[u] == p:
        # hide login, show app, show username in header
        return (
            gr.update(visible=False),
            gr.update(visible=True),
            f"✅ Logged in as: <b>{u}</b>",
            u
        )
    return (
        gr.update(visible=True),
        gr.update(visible=False),
        "❌ Invalid username or password.",
        ""
    )

def do_logout():
    # show login again, hide app, clear header/user
    return (
        gr.update(visible=True),
        gr.update(visible=False),
        "",
        ""
    )

# =========================================================
# 1) Diagnose wrapper (requires your Step-7 diagnose())
# =========================================================
def safe_diagnose(symptoms, obd_text):
    if "diagnose" not in globals():
        return (
            "❌ diagnose() function not found.\n\n"
            "Paste your Step-7 diagnose(symptoms, obd_text) code ABOVE this UI block."
        )
    return diagnose(symptoms, obd_text)  # noqa


# =========================================================
# 2) OBD DATABASE (can be loaded from CSV)
# CSV format expected: columns -> code, description
# =========================================================
OBD_DB = {
    "P0300": "Random/Multiple Cylinder Misfire Detected",
    "P0301": "Cylinder 1 Misfire Detected",
    "P0302": "Cylinder 2 Misfire Detected",
    "P0171": "System Too Lean (Bank 1)",
    "P0420": "Catalyst System Efficiency Below Threshold (Bank 1)",
    "P0128": "Coolant Thermostat (Coolant Temp Below Regulating Temperature)",
    "P0455": "EVAP System Leak Detected (Gross Leak)",
}

def obd_list_text(db):
    codes = sorted(db.keys())
    return "\n".join(codes)

def obd_lookup(code, db):
    c = (code or "").strip().upper()
    if not c:
        return "Enter an OBD code like P0300."
    if c in db:
        return f"{c}: {db[c]}"
    return f"{c}: Not found in loaded OBD list."

def obd_search(query, db):
    q = (query or "").strip().upper()
    if not q:
        return obd_list_text(db)
    matches = []
    for k, v in db.items():
        if q in k or q in v.upper():
            matches.append(k)
    return "\n".join(sorted(matches)) if matches else "No matches found."

def load_obd_csv(file_obj):
    """
    User uploads a CSV with at least: code, description
    Returns: (status_text, db_dict, codes_list_text)
    """
    if file_obj is None:
        return "No file uploaded.", OBD_DB, obd_list_text(OBD_DB)

    path = file_obj.name if hasattr(file_obj, "name") else str(file_obj)
    try:
        df = pd.read_csv(path)
        # normalize column names
        cols = {c.lower().strip(): c for c in df.columns}
        if "code" not in cols or "description" not in cols:
            return (
                "❌ CSV must contain columns: code, description",
                OBD_DB,
                obd_list_text(OBD_DB)
            )

        code_col = cols["code"]
        desc_col = cols["description"]
        db = {}
        for _, row in df.iterrows():
            code = str(row[code_col]).strip().upper()
            desc = str(row[desc_col]).strip()
            if code and code != "NAN":
                db[code] = desc

        if len(db) < 10:
            return (
                f"⚠️ Loaded {len(db)} codes (seems small). Using it anyway.",
                db,
                obd_list_text(db)
            )
        return (f"✅ Loaded {len(db)} OBD codes from CSV.", db, obd_list_text(db))

    except Exception as e:
        return (f"❌ Failed to read CSV: {e}", OBD_DB, obd_list_text(OBD_DB))


# =========================================================
# 3) WARNING LIGHT IMAGES + EXPLANATIONS
# You can replace image paths with your own later.
# =========================================================
WARNING_LIGHTS = [
    {
        "name": "Check Engine (MIL)",
        "desc": "Engine/emissions fault. If flashing: avoid driving; possible misfire. Scan OBD codes.",
        "img": "/content/drive/MyDrive/mini/content/icons/engine_light.png",  # you can set default images later
    },
    {
        "name": "Oil Pressure",
        "desc": "STOP engine ASAP. Low oil pressure can cause severe damage.",
        "img": "/content/drive/MyDrive/mini/content/icons/oil_pressure.png",
    },
    {
        "name": "Battery/Charging",
        "desc": "Charging system fault. Vehicle may stall. Check alternator/battery.",
        "img": "/content/drive/MyDrive/mini/content/icons/battery_changing.png",
    },
    {
        "name": "Coolant Temperature",
        "desc": "Overheating risk. Stop safely; let cool; check coolant when safe.",
        "img": "/content/drive/MyDrive/mini/content/icons/coolant_temp.png",
    },
    {
        "name": "ABS",
        "desc": "ABS may be disabled. Normal brakes work but reduced control; service soon.",
        "img": "/content/drive/MyDrive/mini/content/icons/abs.png",
    },
    {
        "name": "Airbag/SRS",
        "desc": "Airbag system fault. Safety risk; service recommended.",
        "img": "/content/drive/MyDrive/mini/content/icons/airbag.png",
    },
]

def build_warning_gallery():
    """
    Returns a list of tuples: (image_or_placeholder, label)
    Gradio Gallery needs images; we can use a placeholder
    until you add real images.
    """
    items = []
    for w in WARNING_LIGHTS:
        # Placeholder: use a simple colored block by using None (works but blank).
        # Better: replace with real image paths later.
        items.append((w["img"], w["name"]))
    return items

def warning_explain(evt: gr.SelectData):
    """
    Called when user clicks a gallery item.
    evt.index gives the selected index.
    """
    idx = evt.index
    if idx is None or idx >= len(WARNING_LIGHTS):
        return "Select an icon to see details."
    w = WARNING_LIGHTS[idx]
    return f"**{w['name']}**\n\n{w['desc']}"

# =========================================================
# 4) NAVIGATION (sidebar switching)
# =========================================================
def show_page(page):
    return (
        gr.update(visible=(page=="assistant")),
        gr.update(visible=(page=="obd")),
        gr.update(visible=(page=="lights")),
        gr.update(visible=(page=="common")),
    )

COMMON_PROBLEMS = [
    ("Clicking sound on start", "Weak battery / loose terminals / starter relay"),
    ("Overheating", "Low coolant / fan / thermostat / water pump"),
    ("Grinding brakes", "Worn pads/rotors — inspect urgently"),
    ("Vibration at speed", "Wheel balance / alignment / tire wear"),
    ("Poor mileage", "Tire pressure / air filter / O2 sensor / driving style"),
]

def common_text():
    return "\n\n".join([f"• {t}\n  - {d}" for t, d in COMMON_PROBLEMS])


# =========================================================
# 5) Styling (Professional-ish)
# =========================================================
CSS = """
#titlebar {
  padding: 14px 14px 6px 14px;
  border-bottom: 1px solid #eaeaea;
}
#appname { font-size: 22px; font-weight: 700; }
#loggedin { font-size: 13px; opacity: 0.85; margin-top: 4px; }
.sidebar-btn button {
  width: 100% !important;
  justify-content: flex-start !important;
  font-weight: 600 !important;
}
.small-note { font-size: 12px; opacity: 0.8; }
"""

theme = gr.themes.Soft()

# =========================================================
# 6) BUILD UI
# =========================================================
with gr.Blocks(css=CSS, theme=theme, title="Car Assistant LLM — Dashboard") as app:
    # global state: store username + OBD DB dict
    state_user = gr.State("")
    state_obd_db = gr.State(OBD_DB)

    # ------------- LOGIN PAGE -------------
    login_page = gr.Group(visible=True)
    with login_page:
        gr.Markdown("## 🔐 Login")
        with gr.Row():
            with gr.Column(scale=1):
                in_user = gr.Textbox(label="Username", placeholder="admin")
                in_pass = gr.Textbox(label="Password", type="password", placeholder="car123")
                btn_login = gr.Button("Login")
                login_msg = gr.Markdown("")
            with gr.Column(scale=1):
                gr.Markdown("### Demo Credentials\n- **admin / car123**\n- **sir / demo123**")

    # ------------- MAIN APP -------------
    main_app = gr.Group(visible=False)
    with main_app:
        with gr.Row():
            with gr.Column():
                gr.HTML(
                    """
                    <div id="titlebar">
                      <div id="appname">🚗 Car Assistant LLM — Dashboard</div>
                      <div id="loggedin"> </div>
                    </div>
                    """
                )
        header = gr.HTML("<div id='loggedin'></div>")

        with gr.Row():
            # Sidebar
            with gr.Column(scale=1, min_width=240):
                btn_assistant = gr.Button("🧰  Car Assistant", elem_classes=["sidebar-btn"])
                btn_obd = gr.Button("🔎  OBD Code Database", elem_classes=["sidebar-btn"])
                btn_lights = gr.Button("🚨  Warning Light Icons", elem_classes=["sidebar-btn"])
                btn_common = gr.Button("🛠  Common Problems", elem_classes=["sidebar-btn"])
                gr.Markdown("---")
                btn_logout = gr.Button("🚪 Logout")
                gr.Markdown("<div class='small-note'>Tip: add make/model/year for better accuracy.</div>")

            # Pages
            with gr.Column(scale=4):
                # Page: Assistant
                page_assistant = gr.Group(visible=True)
                with page_assistant:
                    gr.Markdown("### 🧰 Car Assistant (Symptoms → Guidance)")
                    symptoms = gr.Textbox(lines=5, label="Symptoms", placeholder="e.g., Clicking sound when starting, no warning lights")
                    obd_codes = gr.Textbox(lines=1, label="OBD Codes (optional)", placeholder="e.g., P0300, P0171")
                    btn_diag = gr.Button("Diagnose ✅")
                    out_diag = gr.Textbox(lines=16, label="Diagnosis & Guidance")
                    btn_diag.click(fn=safe_diagnose, inputs=[symptoms, obd_codes], outputs=out_diag)

                    gr.Markdown("#### Examples")
                    gr.Examples(
                        examples=[
                            ["Clicking sound when starting, no warning lights", "P0300"],
                            ["Strong raw gas smell and heavy engine shaking", "P0302"],
                            ["Engine overheating, steam near hood", "P0128"],
                            ["Grinding noise while braking, pedal feels soft", ""],
                        ],
                        inputs=[symptoms, obd_codes],
                    )

                # Page: OBD Database
                page_obd = gr.Group(visible=False)
                with page_obd:
                    gr.Markdown("### 🔎 OBD Code Database")
                    gr.Markdown("Upload a CSV with columns: **code, description** to load a full list.")
                    with gr.Row():
                        with gr.Column():
                            obd_csv = gr.File(label="Upload OBD CSV (optional)", file_types=[".csv"])
                            btn_load = gr.Button("Load CSV")
                            load_status = gr.Markdown("")
                        with gr.Column():
                            search_q = gr.Textbox(label="Search (code/keyword)", placeholder="misfire / P03")
                            btn_search = gr.Button("Search")
                    codes_list = gr.Textbox(lines=12, value=obd_list_text(OBD_DB), label="Available Codes")
                    lookup_code = gr.Textbox(label="Lookup code", placeholder="P0302")
                    btn_lookup = gr.Button("Lookup")
                    meaning = gr.Textbox(lines=6, label="Meaning")

                    btn_load.click(fn=load_obd_csv, inputs=[obd_csv], outputs=[load_status, state_obd_db, codes_list])

                    btn_search.click(fn=lambda q, db: obd_search(q, db), inputs=[search_q, state_obd_db], outputs=codes_list)
                    btn_lookup.click(fn=lambda c, db: obd_lookup(c, db), inputs=[lookup_code, state_obd_db], outputs=meaning)

                # Page: Warning lights
                page_lights = gr.Group(visible=False)
                with page_lights:
                    gr.Markdown("### 🚨 Warning Light Icons")
                    gr.Markdown("Click an icon to see explanation. (Replace placeholders with real images later.)")
                    gallery = gr.Gallery(
                        label="Warning Lights",
                        value=build_warning_gallery(),
                        columns=3,
                        height=240
                    )
                    explain_box = gr.Markdown("Select an icon to see details.")
                    gallery.select(fn=warning_explain, inputs=None, outputs=explain_box)

                # Page: Common Problems
                page_common = gr.Group(visible=False)
                with page_common:
                    gr.Markdown("### 🛠 Common Car Problems")
                    common_box = gr.Textbox(lines=18, value=common_text(), label="Quick Reference")

        # Sidebar navigation wiring
        btn_assistant.click(fn=lambda: show_page("assistant"), inputs=None,
                           outputs=[page_assistant, page_obd, page_lights, page_common])
        btn_obd.click(fn=lambda: show_page("obd"), inputs=None,
                     outputs=[page_assistant, page_obd, page_lights, page_common])
        btn_lights.click(fn=lambda: show_page("lights"), inputs=None,
                         outputs=[page_assistant, page_obd, page_lights, page_common])
        btn_common.click(fn=lambda: show_page("common"), inputs=None,
                         outputs=[page_assistant, page_obd, page_lights, page_common])

        # Logout wiring
        btn_logout.click(fn=do_logout, inputs=None, outputs=[login_page, main_app, header, state_user])

    # Login wiring: show username in header nicely + store state_user
    def render_header(u):
        if not u:
            return "<div id='loggedin'></div>"
        return f"<div id='loggedin'>✅ Logged in as: <b>{u}</b></div>"

    btn_login.click(
        fn=do_login,
        inputs=[in_user, in_pass],
        outputs=[login_page, main_app, login_msg, state_user]
    ).then(
        fn=render_header,
        inputs=[state_user],
        outputs=[header]
    )

app.launch(share=True)

/tmp/ipython-input-2079/3025453790.py:236: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=CSS, theme=theme, title="Car Assistant LLM — Dashboard") as app:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f00a538f602c352632.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
